In [1]:
from pyspark.sql import SparkSession
from graphframes import GraphFrame

spark = SparkSession.builder.getOrCreate()

In [2]:
vertexesPath = "Spark/Exercises/Ex_53/vertexes.csv"
edgesPath = "Spark/Exercises/Ex_53/edges.csv"
outputPath = "Spark/Exercises/Ex_53/res"

In [3]:
vertexesDF = spark.read.load(path=vertexesPath , format="csv", header=True, inferSchema=True)
vertexesDF.show()

+---+-----+---+
| id| name|age|
+---+-----+---+
| u1|Alice| 34|
| u2|  Bob| 36|
| u3| John| 30|
| u4|David| 29|
| u5| Paul| 32|
| u6| Adel| 36|
| u7| Eddy| 60|
+---+-----+---+



In [4]:
edgesDF = spark.read.load(path=edgesPath , format="csv", header=True, inferSchema=True)
edgesDF.show()

+---+---+--------+
|src|dst|linktype|
+---+---+--------+
| u1| u2|  friend|
| u1| u4|  friend|
| u1| u5|  friend|
| u2| u1|  friend|
| u2| u3|  follow|
| u3| u2|  follow|
| u4| u1|  friend|
| u4| u5|  friend|
| u5| u1|  friend|
| u5| u4|  friend|
| u5| u6|  follow|
| u6| u3|  follow|
| u7| u6|  follow|
+---+---+--------+



In [8]:
filteredEdgesDF = edgesDF.filter("linktype == 'follow'")

In [9]:
g = GraphFrame(vertexesDF, filteredEdgesDF)

In [10]:
followersCountDF = g.inDegrees
followersCountDF.show()

+---+--------+
| id|inDegree|
+---+--------+
| u3|       2|
| u6|       2|
| u2|       1|
+---+--------+



In [19]:
maxFolDF = followersCountDF.agg({"inDegree":"max"}).withColumnRenamed("max(inDegree)","maxNumFollowers")
maxFolDF.show()

+---------------+
|maxNumFollowers|
+---------------+
|              2|
+---------------+



In [32]:
maxFol = maxFolDF.first().maxNumFollowers
maxFol

2

In [33]:
finalDF = followersCountDF.filter("inDegree =="+str(maxFol)).withColumnRenamed("inDegree","numFollowers")
finalDF.show()

+---+------------+
| id|numFollowers|
+---+------------+
| u3|           2|
| u6|           2|
+---+------------+



In [35]:
finalDF.write.csv(outputPath,header=True)